In [16]:
import numpy as np
import requests
import pandas as pd
%matplotlib inline

In [3]:
np.load('rainbow.npy')

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [4]:
df = pd.read_json('apple.ndjson', lines=True)

In [5]:
df_unrecognized = df[df['recognized'] == False]

In [6]:
df_unrecognized['countrycode'].value_counts().head()

US    2480
GB     384
CA     166
KR     123
SA     123
Name: countrycode, dtype: int64

In [45]:
(df_unrecognized.groupby('countrycode')
 .filter(lambda x: x['countrycode'].count() > 20)['countrycode']
 .value_counts())

US    2480
GB     384
CA     166
SA     123
KR     123
AU     100
RU      89
DE      81
SE      73
PH      65
FI      56
CZ      54
IT      52
TW      48
FR      45
PL      45
TH      41
ID      41
AE      40
BR      40
IE      31
HU      31
RO      31
NL      28
IN      26
NZ      23
VN      22
JP      21
BG      21
MY      21
Name: countrycode, dtype: int64

In [48]:
len(df[df['countrycode'] == 'CH'])

510

In [8]:
df_unrecognized['word'].value_counts()

apple    4820
Name: word, dtype: int64

In [51]:
df_unrecognized.ix[0]['drawing']

[[[675], [158], [0]],
 [[675, 675], [158, 158], [185, 189]],
 [[675, 675], [158, 158], [221, 232]],
 [[675, 674], [158, 159], [257, 1018]],
 [[480,
   474,
   468,
   453,
   431,
   407,
   383,
   357,
   333,
   310,
   291,
   280,
   275,
   275,
   283,
   299,
   324,
   353,
   382,
   409,
   434,
   456,
   473,
   486,
   490,
   486,
   466,
   441,
   441],
  [237,
   223,
   218,
   219,
   227,
   238,
   251,
   272,
   297,
   325,
   358,
   393,
   427,
   462,
   497,
   523,
   541,
   549,
   549,
   536,
   511,
   477,
   432,
   384,
   337,
   295,
   259,
   229,
   229],
  [5195,
   5216,
   5258,
   5284,
   5299,
   5315,
   5333,
   5351,
   5366,
   5383,
   5400,
   5417,
   5433,
   5450,
   5467,
   5484,
   5501,
   5516,
   5534,
   5550,
   5566,
   5586,
   5613,
   5616,
   5642,
   5654,
   5664,
   5681,
   5698]],
 [[407,
   402,
   396,
   396,
   402,
   412,
   421,
   431,
   444,
   457,
   468,
   480,
   491,
   504,
   516,
   527,
   

In [22]:

# Copyright 2017 Google Inc.
# 
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
# https://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import struct
from struct import unpack


def unpack_drawing(file_handle):
    key_id, = unpack('Q', file_handle.read(8))
    countrycode, = unpack('2s', file_handle.read(2))
    recognized, = unpack('b', file_handle.read(1))
    timestamp, = unpack('I', file_handle.read(4))
    n_strokes, = unpack('H', file_handle.read(2))
    image = []
    for i in range(n_strokes):
        n_points, = unpack('H', file_handle.read(2))
        fmt = str(n_points) + 'B'
        x = unpack(fmt, file_handle.read(n_points))
        y = unpack(fmt, file_handle.read(n_points))
        image.append((x, y))

    return {
        'key_id': key_id,
        'countrycode': countrycode,
        'recognized': recognized,
        'timestamp': timestamp,
        'image': image
    }


def unpack_drawings(filename):
    with open(filename, 'rb') as f:
        while True:
            try:
                yield unpack_drawing(f)
            except struct.error:
                break

df_unrec = []
df = []
for drawing in unpack_drawings('face.bin'):
    # do something with the drawing
    df.append(drawing)
    if drawing['recognized'] == False:
        df_unrec.append(drawing)

In [23]:
df_unrec = pd.DataFrame(df_unrec)
df = pd.DataFrame(df)

In [49]:
df.groupby('countrycode')['recognized'].value_counts()

countrycode  recognized
b'AE'        1               938
             0                70
b'AF'        1                 1
b'AG'        1                 3
b'AL'        1                37
             0                 3
b'AM'        1                28
             0                 3
b'AN'        1                 1
b'AO'        1                 1
b'AR'        1               333
             0                34
b'AT'        1               651
             0                30
b'AU'        1              3292
             0               322
b'AW'        1                 5
             0                 2
b'AX'        1                18
             0                 1
b'AZ'        1                29
             0                 1
b'BA'        1               281
             0                31
b'BB'        1                 6
b'BD'        1                81
             0                 5
b'BE'        1               496
             0                11
b'BG'        1     

In [55]:
(df.groupby('countrycode')
 .filter(lambda x: len(x) > 1000)
 .groupby('countrycode')
 .apply(lambda x: x[x['recognized'] == 0]['recognized'].count() / len(x) * 100)
 .sort_values(ascending=False))

countrycode
b'TH'    14.186851
b'HU'    11.942594
b'PH'    11.869748
b'ID'     9.527221
b'US'     9.356876
b'AU'     8.909795
b'GB'     8.721817
b'CA'     8.676366
b'IT'     8.390023
b'VN'     8.284519
b'IN'     7.874618
b'CZ'     7.692308
b'RO'     7.611153
b'FI'     7.508044
b'SE'     7.469459
b'AE'     6.944444
b'SK'     6.488550
b'PL'     6.345067
b'FR'     5.959964
b'RU'     5.957556
b'SA'     5.699482
b'NL'     5.451630
b'BR'     4.755481
b'TW'     4.598826
b'DE'     4.268406
b'JP'     3.578337
b'KR'     2.241379
dtype: float64